In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pandas numpy scikit-learn nibabel tensorflow

In [ ]:
# Data handling and preprocessing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# MRI file handling with FreeSurfer-compatible library
import nibabel as nib

# Deep learning model with TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Conv3D, MaxPooling3D, concatenate, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Conv3D, MaxPooling3D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
# Load demographic data
demographic_data = pd.read_excel('/content/drive/MyDrive/Dementia Hunters /mri/oasis_cross-sectional.xlsx')  # Replace with actual path

# Select numeric columns for scaling
numeric_columns = ['Age', 'Educ', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
demographic_data_numeric = demographic_data[numeric_columns]

# Fill NaN values and scale numeric features
demographic_data_numeric.fillna(demographic_data_numeric.mean(), inplace=True)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(demographic_data_numeric)

# Prepare target variable (1 if Alzheimer's present, 0 otherwise)
y = (demographic_data['CDR'].fillna(0) > 0).astype(int).values

# Split the demographic and target data
X_demo_train, X_demo_test, y_train, y_test, idx_train, idx_test = train_test_split(
    scaled_features, y, np.arange(len(y)), test_size=0.2, random_state=42
)

<ipython-input-5-4463f3163b8d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_data_numeric.fillna(demographic_data_numeric.mean(), inplace=True)


In [ ]:
# Model Definition
def create_model(input_shape_mri, input_shape_demo):
    # Define CNN for brain imaging data
    input_img = Input(shape=input_shape_mri, name='mri_input')
    x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu')(input_img)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Conv3D(64, kernel_size=(3, 3, 3), activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = Flatten()(x)

    # Define dense network for demographic data
    input_demo = Input(shape=input_shape_demo, name='demo_input')
    y = Dense(64, activation='relu')(input_demo)
    y = Dropout(0.5)(y)
    y = Dense(32, activation='relu')(y)

    # Combine both networks
    combined = concatenate([x, y])
    z = Dense(64, activation='relu')(combined)
    z = Dense(32, activation='relu')(z)
    z = Dense(1, activation='sigmoid')(z)  # Output layer for binary classification

    # Define the model
    model = Model(inputs=[input_img, input_demo], outputs=z)
    return model


In [ ]:
# Save the trained model to disk
model_path = "/content/drive/MyDrive/Dementia_Hunters/saved_model"
model.save(model_path)

# Clear the model from memory to free up resources
del model
tf.keras.backend.clear_session()


In [ ]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class MRIDataGenerator(Sequence):
    def __init__(self, file_paths, demo_data, labels, batch_size=1, dim=(256, 256, 256), n_channels=1, shuffle=True):
        self.file_paths = file_paths
        self.demo_data = demo_data
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.indices = np.arange(len(self.file_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_img, X_demo, y = self.__data_generation(batch_indices)
        return (tf.convert_to_tensor(X_img, dtype=tf.float32), tf.convert_to_tensor(X_demo, dtype=tf.float32)), tf.convert_to_tensor(y, dtype=tf.float32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        X_img = np.empty((self.batch_size, *self.dim, self.n_channels))
        X_demo = np.empty((self.batch_size, self.demo_data.shape[1]))
        y = np.empty((self.batch_size), dtype=int)

        for i, idx in enumerate(batch_indices):
            file_path = self.file_paths[idx]
            # Load MRI data
            mgz_file = nib.load(file_path)
            data = mgz_file.get_fdata()
            X_img[i, ] = np.expand_dims(data, axis=-1)

            X_demo[i, ] = self.demo_data[idx]
            y[i] = self.labels[idx]

        return X_img, X_demo, y

# Prepare your data
def prepare_data(idx_list, demo_data, labels, base_path):
    file_paths = []
    demo_data_filtered = []
    labels_filtered = []

    for pos, idx in enumerate(idx_list):
        file_path = os.path.join(base_path, f'aseg{idx + 1:02d}.mgz')
        if os.path.exists(file_path):
            file_paths.append(file_path)
            demo_data_filtered.append(demo_data[pos])
            labels_filtered.append(labels[pos])

    demo_data_filtered = np.array(demo_data_filtered)
    labels_filtered = np.array(labels_filtered)

    return file_paths, demo_data_filtered, labels_filtered

# Base path to your MRI files
base_path = '/content/drive/MyDrive/Dementia Hunters /mri/'

# Prepare training data
train_file_paths, train_demo_data_filtered, train_labels_filtered = prepare_data(
    idx_train, X_demo_train, y_train, base_path
)

# Prepare testing data
test_file_paths, test_demo_data_filtered, test_labels_filtered = prepare_data(
    idx_test, X_demo_test, y_test, base_path
)

# Create generators
train_generator = MRIDataGenerator(
    file_paths=train_file_paths,
    demo_data=train_demo_data_filtered,
    labels=train_labels_filtered,
    batch_size=1
)

test_generator = MRIDataGenerator(
    file_paths=test_file_paths,
    demo_data=test_demo_data_filtered,
    labels=test_labels_filtered,
    batch_size=1,
    shuffle=False
)

# Create the model
input_shape_mri =  (256, 256, 256, 1)
input_shape_demo = (train_demo_data_filtered.shape[1],)
model = create_model(input_shape_mri, input_shape_demo)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    x=train_generator,
    validation_data=test_generator,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


292/292 ━━━━━━━━━━━━━━━━━━━━ 524s 2s/step - accuracy: 0.6736 - loss: 243.2444 - val_accuracy: 0.7949 - val_loss: 0.4016
Epoch 2/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 174s 584ms/step - accuracy: 0.9636 - loss: 0.0943 - val_accuracy: 0.8462 - val_loss: 0.3893
Epoch 3/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 173s 583ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.8462 - val_loss: 0.4449
Epoch 4/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 174s 583ms/step - accuracy: 1.0000 - loss: 4.1427e-04 - val_accuracy: 0.8718 - val_loss: 0.5616
Epoch 5/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 174s 583ms/step - accuracy: 1.0000 - loss: 4.2000e-04 - val_accuracy: 0.8590 - val_loss: 0.5047
Epoch 6/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 173s 582ms/step - accuracy: 1.0000 - loss: 1.5804e-04 - val_accuracy: 0.8590 - val_loss: 0.5342
Epoch 7/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 170s 571ms/step - accuracy: 1.0000 - loss: 1.3739e-04 - val_accuracy: 0.8718 - val_loss: 0.5517
Epoch 8/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 174s 584ms/step - accuracy: 1.00

In [ ]:
train_file_paths, train_demo_data_filtered, train_labels_filtered

(['/content/drive/MyDrive/Dementia Hunters /mri/aseg266.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg406.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg32.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg85.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg300.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg174.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg95.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg228.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg429.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg153.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg355.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg273.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg419.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg06.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg46.mgz',
  '/content/drive/MyDrive/Dementia Hunters /mri/aseg428.mgz',
  '/content/d

In [ ]:
import shap
import matplotlib.pyplot as plt
import numpy as np

def explain_padded_slice_with_shap(model, mri_scan, demo_data, slice_index):
    """
    Explains a single slice of an MRI scan by padding it to match the model's input shape.
    """
    # Extract the MRI data input shape dynamically
    mri_input_shape = model.input_shape[0]  # (None, height, width, depth, channels)
    height, width, depth, channels = mri_input_shape[1], mri_input_shape[2], mri_input_shape[3], mri_input_shape[4]

    # Extract a single 2D slice (e.g., axial plane)
    mri_slice = mri_scan[:, :, slice_index]  # Shape: (height, width)

    # Add channel dimension (for grayscale)
    mri_padded = np.expand_dims(mri_slice, axis=-1)  # Shape: (height, width, 1)

    # Pad the depth dimension to match the model's input depth
    mri_padded = np.pad(mri_padded, ((0, 0), (0, 0), (0, depth - 1)))  # Shape: (height, width, depth)

    # Add batch dimension
    mri_padded = np.expand_dims(mri_padded, axis=0)  # Shape: (1, height, width, depth, channels)

    # Add batch dimension for demographic data
    demo_data = np.expand_dims(demo_data, axis=0)  # Shape: (1, demo_features)

    # Create a SHAP explainer for multi-input models
    explainer = shap.GradientExplainer(model, [mri_padded, demo_data])

    # Compute SHAP values
    shap_values = explainer.shap_values([mri_padded, demo_data])
    return mri_padded.squeeze(), shap_values[0].squeeze()

def plot_padded_shap_slice(mri_padded, shap_values):
    """
    Visualizes the SHAP explanations on a padded MRI slice.
    """
    # Reduce back to the original slice shape for visualization
    mri_slice = mri_padded[:, :, 0]  # Extract the slice from the padded volume
    shap_slice = shap_values[:, :, 0]

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(mri_slice, cmap="gray")
    ax[0].set_title("Original MRI Slice")
    ax[1].imshow(mri_slice, cmap="gray")
    ax[1].imshow(shap_slice, cmap="hot", alpha=0.5)
    ax[1].set_title("SHAP Explanations Overlay")
    plt.show()

# Process a single scan from the generator
(X_img_batch, X_demo_batch), _ = test_generator[0]
mri_scan = X_img_batch[0].numpy()  # Shape: (height, width, depth, channels)
demo_data = X_demo_batch[0].numpy()  # Shape: (8,)

# Remove the channel dimension for simplicity
mri_scan = mri_scan.squeeze()  # Shape: (height, width, depth)

# Select a slice to explain (e.g., middle axial slice)
slice_index = mri_scan.shape[2] // 2

# Explain the padded slice using SHAP
mri_padded, shap_values = explain_padded_slice_with_shap(model, mri_scan, demo_data, slice_index)

# Visualize the SHAP explanations on the padded slice
plot_padded_shap_slice(mri_padded, shap_values)


In [ ]:
import tensorflow as tf
import numpy as np
import shap
import matplotlib.pyplot as plt

# Load the saved model
model_path = "/content/drive/MyDrive/Dementia_Hunters/saved_model"
model = tf.keras.models.load_model(model_path)

# Process a single scan from the generator (ensure test_generator is re-initialized)
(X_img_batch, X_demo_batch), _ = test_generator[0]

# Extract a single MRI slice
mri_scan = X_img_batch[0].numpy().squeeze()  # Shape: (256, 256, 256)
demo_data = X_demo_batch[0].numpy()          # Shape: (demo_features,)

# Select a slice to visualize (e.g., middle axial slice)
slice_index = mri_scan.shape[2] // 2
mri_slice = mri_scan[:, :, slice_index]

# Add channel and batch dimensions for compatibility with the model
mri_slice = np.expand_dims(np.expand_dims(mri_slice, axis=-1), axis=0)  # Shape: (1, 256, 256, 1)
demo_data = np.expand_dims(demo_data, axis=0)  # Shape: (1, demo_features)

# Use SHAP for visualizations
explainer = shap.DeepExplainer(model, [mri_slice, demo_data])
shap_values = explainer.shap_values([mri_slice, demo_data])

# Visualize the results
def plot_shap_on_slice(mri_slice, shap_values):
    """
    Plots a single MRI slice with SHAP explanations.
    """
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(mri_slice.squeeze(), cmap="gray")
    ax[0].set_title("Original MRI Slice")
    ax[1].imshow(mri_slice.squeeze(), cmap="gray")
    ax[1].imshow(shap_values[0].squeeze(), cmap="hot", alpha=0.5)
    ax[1].set_title("SHAP Explanations Overlay")
    plt.show()

plot_shap_on_slice(mri_slice, shap_values)


NameError: name 'model' is not defined